<a href="https://colab.research.google.com/github/tyukei/translate-ipynb/blob/main/translate_en2ja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.4 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=30bfc8b2ebac053aab841df9ee7fd9758c6b1a5a1b3817e71cfbfe03b0c006e3
  Stored in directory: /root/.cache/pip/wheels/39/17/6f/66a045ea3d168826074691b4b787b8f324d3f646d755443fda
Successfully built googletrans
  Attempting uninstall: hyperframe
    Found existing installation: hyperframe 6.1.0
    Uninstalling hyperfram

In [ ]:
import json
import requests
from googletrans import Translator

def convert_to_raw_url(url):
    """
    GitHub上の通常の.ipynbファイルURLをrawファイルのURLに変換する
    例: https://github.com/user/repo/blob/branch/notebook.ipynb →
         https://raw.githubusercontent.com/user/repo/branch/notebook.ipynb
    """
    if "github.com" in url and "/blob/" in url:
        raw_url = url.replace("github.com", "raw.githubusercontent.com").replace("/blob/", "/")
        return raw_url
    return url

def translate_markdown_in_ipynb(github_url, output_filename):
    # URLをrawファイルのURLに変換
    raw_url = convert_to_raw_url(github_url)

    # GitHubから.ipynbファイルをダウンロード
    response = requests.get(raw_url)
    response.raise_for_status()  # エラーがあれば例外を発生
    notebook = response.json()

    translator = Translator()

    # 各セルをチェックし、Markdownセルであれば翻訳
    for cell in notebook.get("cells", []):
        if cell.get("cell_type") == "markdown":
            # セル内のテキストは通常リスト形式なので、結合して一つの文字列にする
            original_text = "".join(cell.get("source", []))
            if original_text.strip():  # 空文字列でなければ
                translated = translator.translate(original_text, src='en', dest='ja')
                translated_text = translated.text
                # 翻訳結果をセルの内容に置換（リスト形式で保存）
                cell["source"] = [translated_text]

    # 翻訳済みノートブックを新たなファイルに保存
    with open(output_filename, "w", encoding="utf-8") as f:
        json.dump(notebook, f, ensure_ascii=False, indent=2)
    print(f"翻訳済みノートブックは {output_filename} に保存されました。")

if __name__ == "__main__":
    # GitHub上の.ipynbファイルのURLを入力
    github_url = input("GitHub上の.ipynbファイルのURLを入力してください: ").strip()
    output_filename = "translated_notebook.ipynb"
    translate_markdown_in_ipynb(github_url, output_filename)
